# CARLA Exploration
This is an exploration of all the options that are offered to us out of the box to generate synthetic data.

In [2]:
import carla
import random
import cv2
import skimage.measure as measure

#in synchronous mode, sensor data must be added to a queue
import queue

In [3]:
client = carla.Client('carla_server', 2000)
client.set_timeout(60.0)

In [4]:
# Let's see what Maps are currently offered to us
maps = client.get_available_maps()
sorted(maps)

['/Game/Carla/Maps/Town01',
 '/Game/Carla/Maps/Town01_Opt',
 '/Game/Carla/Maps/Town02',
 '/Game/Carla/Maps/Town02_Opt',
 '/Game/Carla/Maps/Town03',
 '/Game/Carla/Maps/Town03_Opt',
 '/Game/Carla/Maps/Town04',
 '/Game/Carla/Maps/Town04_Opt',
 '/Game/Carla/Maps/Town05',
 '/Game/Carla/Maps/Town05_Opt',
 '/Game/Carla/Maps/Town10HD',
 '/Game/Carla/Maps/Town10HD_Opt']

What is the difference between `Town01` and `Town01_Opt`? The "Opt" maps are layered maps, `Town01_Opt` has extra props (roads, sidewalks, traffic lights and traffic signs). Learn more here: https://carla.readthedocs.io/en/latest/core_map/#landmarks

![](https://carla.readthedocs.io/en/latest/img/sublevels.gif)

In [5]:
world = client.load_world('Town04')
settings = world.get_settings()
settings.fixed_delta_seconds = 0.05 #must be less than 0.1, or else physics will be noisy
#must use fixed delta seconds and synchronous mode for python api controlled sim, or else 
#camera and sensor data may not match simulation properly and will be noisy 
settings.synchronous_mode = True 
world.apply_settings(settings)

3

In [4]:
weather = carla.WeatherParameters(
    cloudiness=20.0,
    precipitation=20.0,
    sun_altitude_angle=110.0)

#or use precomputed weathers
#weather = carla.WeatherParameters.WetCloudySunset

world.set_weather(weather)

In [5]:
blueprints = world.get_blueprint_library().filter('*')
for blueprint in random.sample(list(blueprints), 5):
    print(blueprint.id)
    for attr in blueprint:
       print('  - {}'.format(attr))

static.prop.creasedbox02
  - ActorAttribute(id=size,type=str,value=small(const))
  - ActorAttribute(id=role_name,type=str,value=prop)
vehicle.bh.crossbike
  - ActorAttribute(id=generation,type=int,value=1(const))
  - ActorAttribute(id=number_of_wheels,type=int,value=2(const))
  - ActorAttribute(id=sticky_control,type=bool,value=True)
  - ActorAttribute(id=driver_id,type=int,value=2)
  - ActorAttribute(id=object_type,type=str,value=(const))
  - ActorAttribute(id=color,type=Color,value=Color(20,78,217,255))
  - ActorAttribute(id=role_name,type=str,value=autopilot)
vehicle.carlamotors.carlacola
  - ActorAttribute(id=generation,type=int,value=1(const))
  - ActorAttribute(id=number_of_wheels,type=int,value=4(const))
  - ActorAttribute(id=sticky_control,type=bool,value=True)
  - ActorAttribute(id=object_type,type=str,value=(const))
  - ActorAttribute(id=color,type=Color,value=Color(255,68,0,255))
  - ActorAttribute(id=role_name,type=str,value=autopilot)
vehicle.bmw.grandtourer
  - ActorAttri

In [6]:
actor_list = []

In [7]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle')) # lets choose a vehicle at random

# lets choose a random spawn point
transform = random.choice(world.get_map().get_spawn_points()) 

#spawn a vehicle
vehicle = world.spawn_actor(bp, transform) 
actor_list.append(vehicle)

vehicle.set_autopilot(True)

In [8]:
#lets create waypoints for driving the vehicle around automatically
m = world.get_map()
waypoint = m.get_waypoint(transform.location)

#lets add more vehicles
for _ in range(0, 200):
    transform = random.choice(m.get_spawn_points())

    bp_vehicle = random.choice(blueprint_library.filter('vehicle'))

    # This time we are using try_spawn_actor. If the spot is already
    # occupied by another object, the function will return None.
    other_vehicle = world.try_spawn_actor(bp_vehicle, transform)
    if other_vehicle is not None:
        #print(npc)
        other_vehicle.set_autopilot(True)
        actor_list.append(other_vehicle)

In [9]:
# Adding random objects
blueprint_library = world.get_blueprint_library()
weirdobj_bp = blueprint_library.find('static.prop.fountain')
weirdobj_transform = random.choice(world.get_map().get_spawn_points())
weirdobj_transform = carla.Transform(carla.Location(x=230, y=195, z=40), carla.Rotation(yaw=180))
weird_obj = world.try_spawn_actor(weirdobj_bp, weirdobj_transform)
actor_list.append(weird_obj)